# SteganoGAN in Keras
This notebook contains code attempting to reimplement SteganoGAN in Keras, for the purpose of better understanding (and scrutinizing) it.

*Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan*

### Modules

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import sys
sys.path.append("../..")
sys.path.append("../../..")

import tensorflow as tf
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import CSVLogger
from callbacks import Checkpoint, SaveImages

from models import (
  steganogan_encoder_basic_model,
  steganogan_decoder_basic_model
)

from dataset_utils import normalize_img, create_message_dataset
from keras_steganogan import KerasSteganoGAN

2024-09-22 14:43:10.822613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Constants

In [2]:
# Image dimensions
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3

IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
MESSAGE_DEPTH = 1
BATCH_SIZE = 4
MODEL_PATH = f'../../pretrained_models/{MESSAGE_DEPTH}/steganoGAN_basic.keras'
LOGS_PATH = f'../../logs/basic/{MESSAGE_DEPTH}/steganoGAN.csv'
CALLBACK_IMAGES_PATH = '../../../images/callback'
CALLBACK_IMAGES_OUTPUT_PATH = f'../../epoch_images/basic/{MESSAGE_DEPTH}'

----

### Build model for future train

In [3]:
encoder = steganogan_encoder_basic_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, MESSAGE_DEPTH)
decoder = steganogan_decoder_basic_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, MESSAGE_DEPTH)

steganoGAN = KerasSteganoGAN(
  encoder=encoder,
  decoder=decoder,
  image_shape=IMAGE_SHAPE,
  data_depth=MESSAGE_DEPTH,
  model_path=MODEL_PATH
)

steganoGAN.compile(
  encoder_optimizer = Adam(learning_rate=1e-4, beta_1=0.5),
  decoder_optimizer = Adam(learning_rate=1e-4, beta_1=0.5),
  loss_fn = BinaryCrossentropy(from_logits=False)
)

#steganoGAN.models_summary()
#steganoGAN.summary()

### Download div2k dataset and complete it with random message dataset of {0, 1}

In [4]:
train_dir = '/Users/dmitryhoma/Projects/phd_dissertation/state_2/SteganoGAN/research/data/div2k/train'
val_dir = '/Users/dmitryhoma/Projects/phd_dissertation/state_2/SteganoGAN/research/data/div2k/val'

train_image_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    seed=123,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle=True
)

val_image_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    label_mode=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    seed=123,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle=True
)

train_image_ds = train_image_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
val_image_ds = val_image_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

train_message_ds = create_message_dataset(BATCH_SIZE, len(train_image_ds), IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH)
val_message_ds = create_message_dataset(BATCH_SIZE, len(val_image_ds), IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH)

train_ds = tf.data.Dataset.zip((train_image_ds, train_message_ds)).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.zip((val_image_ds, val_message_ds)).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 800 files.
Found 100 files.


In [5]:
steganoGAN.build([(1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), (1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH)])
steganoGAN.fit(train_ds, epochs=40, validation_data=val_ds, callbacks=[
  SaveImages(MESSAGE_DEPTH, IMAGE_SHAPE, CALLBACK_IMAGES_PATH, CALLBACK_IMAGES_OUTPUT_PATH),
  Checkpoint(MODEL_PATH),
  CSVLogger(LOGS_PATH)
])

Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 112s 531ms/step - decoder_accuracy: 0.6240 - decoder_loss: 0.6789 - encoder_decoder_total_loss: 0.9872 - psnr: 5.9105 - rs_bpp: 0.2481 - similarity_loss: 0.3083 - ssim: 0.1894 - val_decoder_accuracy: 0.9498 - val_decoder_loss: 0.3947 - val_encoder_decoder_total_loss: 0.5626 - val_psnr: 7.9004 - val_rs_bpp: 0.8997 - val_similarity_loss: 0.1679 - val_ssim: 0.1661
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 110s 543ms/step - decoder_accuracy: 0.9362 - decoder_loss: 0.1846 - encoder_decoder_total_loss: 0.3004 - psnr: 9.6009 - rs_bpp: 0.8724 - similarity_loss: 0.1158 - ssim: 0.2807 - val_decoder_accuracy: 0.9862 - val_decoder_loss: 0.0983 - val_encoder_decoder_total_loss: 0.1732 - val_psnr: 11.4059 - val_rs_bpp: 0.9725 - val_similarity_loss: 0.0749 - val_ssim: 0.3590
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 113s 561ms/step - decoder_accuracy: 0.9847 - decoder_loss: 0.0540 - encoder_decoder_total_loss: 0.1395 - psnr: 10.9913 - rs_bpp: 0.9693 - similarity_loss: